<a href="https://colab.research.google.com/github/predictions01/zabbix/blob/andrea/prediction_singola_macchina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

prova 3

In [ ]:
 from google.colab import drive
 drive.mount('/content/drive')

In [ ]:
pip install lime shap eli5

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
%run drive/MyDrive/Colab_Notebooks/AJ_draw.py
%run drive/MyDrive/Colab_Notebooks/AJ_ML_eda.py
%run drive/MyDrive/Colab_Notebooks/AJ_models_classifier.py

In [ ]:
main_folder = 'drive/MyDrive/Colab_Notebooks/dati/'
# metrics = pd.read_pickle(main_folder+"zabbix-new2")
# tickets = pd.read_pickle(main_folder+"tickets-new2")

metrics = pd.read_pickle(main_folder+"zabbix_AJ")
tickets = pd.read_pickle(main_folder+"tickets_AJ")

In [ ]:
metrics.to_csv('zabbix_AJ_table.csv')
tickets.to_csv('tickets_AJ_table.csv')

In [ ]:
device = 'SQLDBONL10'
my_metrics = metrics[metrics.device == device].set_index('timestamp')[['metric', 'value']]
df = my_metrics.pivot(columns='metric', values='value').dropna(thresh=my_metrics.index.nunique() * .9, axis=1).interpolate(limit_direction='both')
df['hour'] = df.index.hour
df['day'] = df.index.dayofweek

In [ ]:
my_tickets = tickets[tickets.device == device].set_index('timestamp')

my_tickets2 = my_tickets.copy()
col_my_tickets2 = my_tickets2.columns.tolist()
my_tickets2['y'] = 1
my_tickets2 = my_tickets2.drop(col_my_tickets2, axis=1).reset_index()
my_tickets2['timestamp'] = my_tickets2['timestamp'].dt.round(freq = '1h')
df = df.reset_index()
df_new = pd.merge(df, my_tickets2, on = 'timestamp', how = 'outer')
df_new['y'] = df_new['y'].fillna(0)
df_new.set_index('timestamp', inplace=True)

hours_pre_event = 12
hour_post_event = 0

def increase_events_calculator(df, hours, direction = 1):
    hours = hours + 1
    for i in range(1,hours):
        df['y'+str(i)] = df['y'].shift(i*direction)

    for i in range(1,hours):
        df['y'] = df['y']+df['y'+str(i)]

    df['y'] = df['y'].fillna(0)
    dropy = ['y'+str(i) for i in range(1,hours)]
    df = df.drop(dropy, axis = 1)
    return df

df_new = increase_events_calculator(df_new, hour_post_event)
df_new = increase_events_calculator(df_new, hours_pre_event, direction = -1)
df_new['y'] = df_new['y'].replace([i for i in range(1,int(df_new['y'].max()+1))], 1).astype(int)

df = df_new

In [ ]:
peeking(df).plot_correlation_roy('y')

In [ ]:
correlation_with_target, correlation_within_features = learning(df).correlation_matrix('y', corr_value_w_targhet = 1)

In [ ]:
correlation_with_target

In [ ]:
correlation_within_features

In [ ]:
ds().nuova_fig(1)
ds().dati(x = np.arange(df.shape[0]), y = df['Used memory %'].to_numpy())
# ds().dati(x = np.arange(df.shape[0]), y = df['Free memory'].to_numpy()*30, colore='red')
# ds().range_plot(bottomX = 500, topX = 1000)
ds().porta_a_finestra()

ds().nuova_fig(2)
# ds().dati(x = np.arange(df.shape[0]), y = df['Used memory %'].to_numpy())
ds().dati(x = np.arange(df.shape[0]), y = df['crif_capacity_mem - Used (active+wired)'].to_numpy()*30, colore='red')
# ds().range_plot(bottomX = 500, topX = 1000)
ds().porta_a_finestra()

In [ ]:
def balanced_train_test_split(df, test_size=.7):
    # Calcolo quant'è il 70% del numero di "1"
    num_tickets_training = int(df.y.sum() * test_size)

    # Faccio cumulative sum per considerare nel training set un numero di ticket = num_tickets_training
    cumsum = df.y.cumsum()

    # Conto quante righe ho che stanno dopo i primi <num_tickets_training> ticket
    num_val_tickets = cumsum[cumsum > num_tickets_training].shape[0]

    # Splitto con test_size fissato
    training_set, validation_set = train_test_split(df, test_size=num_val_tickets, shuffle=False)

    return training_set, validation_set

#replace the simbols not compatible with xgboost from the columns names
import re
regex = re.compile(r"\[|\]|<", re.IGNORECASE)
df.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in df.columns.values]

training_set, test_set = balanced_train_test_split(df)
print(len(test_set[test_set.y == 1]) / len(test_set))

In [ ]:
learn = learning_class()
models = learn.get_models(['RandomForestClassifier', 'XGBClassifier', 'KNeighborsClassifier', 'SVC', 'ExtraTreeClassifier', 'LogisticRegression'])
normal, sparse = 'normal', 'sparse'

models['deep learning ' + normal +' '+ sparse] = learn.get_deep_learning_model(input_dl = training_set.loc[:, training_set.columns != 'y'].shape[1],net_type = normal, loss_type = sparse)
models, _ = learn.train_models(models, training_set.loc[:, training_set.columns != 'y'], training_set.y, shuffle = False, epochs = 30)
prob, pred = learn.prob_matrix_generator(models, test_set.loc[:, test_set.columns != 'y'], num_classes=2)
score = learn.score_models(test_set.y, prob, pred)
score

In [ ]:
learn.plot_roc_curve(prob, test_set.y)

In [ ]:
learn.score_accuracy_recall(pred, test_set.y)